### 다익스트라 최단 경로 알고리즘

- 여러개의 노드가 있을 때, 특정한 노드에서 출발하여 다른 노드로 가는 각각의 최단 경로를 구해주는 알고리즘  
  
- '음의 간선(cost)'이 없을 때 정상적으로 동작  
        
- 각 노드에 대한 현재까지의 최단거리 정보를 1차원 리스트에 저장하며 갱신 >> 그리디 알고리즘

In [4]:
# input
# 무한의 값으로 큰 수 설정
INF = int(1e9)
# node, edge 개수 입력 받기
n, m = map(int, input().split())
# 시작 노드 번호
start = int(input())
# 그래프 입력 받기
graph = [[] for i in range(n+1)]
for _ in range(m):
    a, b, c = map(int, input().rstrip().split())
    graph[a].append((b, c))
# 방문 기록용
visited = [False]*(n+1)
# 거리 기록용
distance = [INF]*(n+1)

6 11
1
1 2 2
1 3 5
1 4 1
2 3 3
2 4 2
3 2 3
3 6 5
4 3 3
4 5 1
5 3 1
5 6 2


In [2]:
### 다익스트라 알고리즘 1
# 방문하지 않은 노드 중에서 최단거리가 짧은 노드의 번호를 반환
def sdist():
    temp = INF
    index = 0
    for i in range(1, n+1):
        if distance[i] < temp and not visited[i]:
            temp = distance[i]
            index = i
    return index
# 최단 경로 계산
def spath(start):
    # 시작 노드에 대해서 초기화
    distance[start] = 0
    visited[start] = True
    # 시작 노드와 연결돠어 있는 노드의 거리 계산
    for x in graph[start]:
        distance[x[0]] = x[1]
    # 시작 노드를 제외한 전체 n-2개의 노드에 대해서 반복(마지막 노드는 생략)
    for _ in range(n-2):
        # 현재까지 방문하지 않고 최단 거리를 가진 노드를 꺼내 방문처리
        node = sdist()
        visited[node] = True
        # 현재 노드와 연결된 다른 노드를 확인
        for j in graph[node]:
            cost = distance[node] + j[1]
            # 앞서 계산한 거리보다 짧은 경우만 업데이트
            if cost < distance[j[0]]:
                distance[j[0]] = cost

# 다익스트라 알고리즘 수행
spath(start)
for i in range(1, n+1):
    if distance[i] == INF:
        print('INFINITY')
    else:
        print(distance[i])

0
2
3
1
2
4


- 방문하지 않았으면서 거리가 가장 짧은 노드를 선택하는 과정이 비효율적 >> heap 자료구조를 사용.  
- 파이썬의 힙 라이브러리는 기본적으로 min priority queue를 사용. max priority queue가 필요한 경우엔 음수를 붙여 사용할 수도 있다.

In [5]:
# 개선된 다익스트라 알고리즘 (heapq 사용)
import heapq
def spath(start):
    q = []
    # 시작 노드를 큐에 삽입하고 거리를 0으로 설정
    heapq.heappush(q, (0, start))
    distance[start] = 0
    # 큐가 빌 때까지 반복
    while q:
        dist, node = heapq.heappop(q)
        # 우선순위 큐에서 뽑아온 노드가 이미 업데이트가 된 경우(큐에서 뽑은 거리보다 distance에 저장된 값이 더 작은 경우) >> 무시
        if distance[node] < dist:
            continue
        # 인접한 노드 탐색
        for i in graph[node]:
            cost = dist + i[1]
            # 기존 거리보다 작으면 업데이트
            if cost < distance[i[0]]:
                distance[i[0]] = cost
                heapq.heappush(q, (cost, i[0]))

# 다익스트라 알고리즘 수행
spath(start)
for i in range(1, n+1):
    if distance[i] == INF:
        print('INFINITY')
    else:
        print(distance[i])

0
2
3
1
2
4


### 플로이드 워셜 알고리즘  
- 모든 지점에서 다른 모든 지점까지의 최단 경로를 모두 구해야 하는 경우  
- 다이나믹 프로그래밍의 형태를 가짐  
- 1번 노드가 있을 때, 1번 노드를 거쳐가는 즉 A노드-1번 노드-B노드로 가는 비용을 확인한 후에 최단 거리를 갱신한다.  
- 이때 A노드와 B노드는 1번 노드를 제외한 n-1개의 노드에서 선택해야 하므로 ${}_{n-1}{\rm P}_{2}$만큼의 경우의 수가 존재한다.  
- 각각의 n개 노드에 대하여 ${}_{n-1}{\rm P}_{2}$번을 반복하므로 전체 시간복잡도는 $O(n^3)$이라고 볼 수 있다.  
- 점화식 : $D_{ab} = min(D_{ab}, D_{ak}+D_{kb})$

In [3]:
### 플로이드 워셜 알고리즘 소스코드
# 무한 설정
INF = int(1e9)
# 노드 개수, 간선 개수
n = int(input())
m = int(input())
# 2차원 리스트를 만들고 모든 값을 무한으로 초기화
graph = [[INF]*(n+1) for _ in range(n+1)]
# 자기 자신에서 자기 자신으로 가는 비용은 0으로 초기화
for i in range(1, n+1):
    for j in range(1, n+1):
        if i == j:
            graph[i][j] = 0
# 주어진 그래프 정보 받아오기
for _ in range(m):
    a, b, c = map(int, input().rstrip().split())
    graph[a][b] = c
# 점화식에 따라 플로이드 워셜 알고리즘 수행
for i in range(1, n+1):
    for j in range(1, n+1):
        for k in range(1, n+1):
            graph[j][k] = min(graph[j][k], graph[j][i] + graph[i][k])
# 수행된 결과를 출력
for a in range(1, n+1):
    for b in range(1, n+1):
        if graph[a][b] == INF:
            print('INFINITY', end = ' ')
        else:
            print(graph[a][b], end = ' ')
    print()

4
7
1 2 4
1 4 6
2 1 3
2 3 7
3 1 5
3 4 4
4 3 2
0 4 8 6 
3 0 7 9 
5 9 0 4 
7 11 2 0 


In [10]:
# 실전문제 2 : 미래도시
# 데이터 입력
n, m = map(int, input().split())
INF = int(1e9)
graph = [[INF]*(n+1) for _ in range(n+1)]
for i in range(1, n+1):
    for j in range(1, n+1):
        if i == j:
            graph[i][j] = 0
for _ in range(m):
    a, b = map(int, input().split())
    graph[a][b] = 1
    graph[b][a] = 1
target2, target1 = map(int, input().split())
# 플로이드 워셜 알고리즘
for i in range(1, n+1):
    for a in range(1, n+1):
        for b in range(1, n+1):
            graph[a][b] = min(graph[a][b], graph[a][i]+graph[i][b])

d1 = graph[1][target1]
d2 = graph[target1][target2]
if d1 == INF or d2 == INF:
    print(-1)
else:
    print(d1+d2)

-1


In [14]:
# 실전 문제 2 : 전보
# 데이터 입력
n, m, c = map(int, input().split())
INF = int(1e9)
graph = [[] for _ in range(n+1)]
for _ in range(m):
    x, y, z = map(int, input().rstrip().split())
    graph[x].append((y, z))
    graph[y].append((x, z))
distance = [INF]*(n+1)
# 다익스트라 알고리즘(우선순위 큐 사용)
import heapq
distance[c] = 0
q = []
heapq.heappush(q, (0, c))
while q:
    dist, node = heapq.heappop(q)
    if distance[node] < dist:
        continue
    for x in graph[node]:
        temp = dist + x[1]
        if distance[x[0]] > temp:
            distance[x[0]] = temp
            heapq.heappush(q, (temp, x[0]))
cnt = 0
time = 0
for i in range(1, n+1):
    if i == c:
        continue
    if i != INF:
        cnt += 1
        time = max(time, distance[i])
print(cnt, time)

2 4


In [19]:
# 백준 1753번 : 최단경로
# 입력
n, m = map(int, input().split())
start = int(input())
graph = [[] for _ in range(n+1)]
for i in range(m):
    a, b, c = map(int, input().rstrip().split())
    graph[a].append((b, c))
INF = int(1e9)
distance = [INF]*(n+1)
### 다익스트라 알고리즘
import heapq
q = []
distance[start] = 0
heapq.heappush(q, (0, start))
while q:
    dist, node = heapq.heappop(q)
    if dist > distance[node]:
        continue
    for x in graph[node]:
        temp = dist + x[1]
        if temp < distance[x[0]]:
            distance[x[0]] = temp
            heapq.heappush(q, (temp, x[0]))
# 정답 출력
for i in range(1, n+1):
    if distance[i] == INF:
        print('INF')
    else:
        print(distance[i])

5 6
1
5 1 1
1 2 2
1 3 3
2 3 4
2 4 5
3 4 6
0
2
3
7
INF


In [27]:
# 백준 11404번 : 플로이드
# 입력
n = int(input())
m = int(input())
INF = int(1e9)
graph = [[INF]*(n+1) for _ in range(n+1)]
for _ in range(m):
    a, b, c = map(int, input().rstrip().split())
    graph[a][b] = min(graph[a][b], c)
for i in range(1, n+1):
    for j in range(1, n+1):
        if i == j:
            graph[i][j] = 0
# 플로이드 워셜 알고리즘
for i in range(1, n+1):
    for j in range(1, n+1):
        for k in range(1, n+1):
            graph[j][k] = min(graph[j][k], graph[j][i]+graph[i][k])
# 출력
for i in range(1, n+1):
    for j in range(1, n+1):
        if graph[i][j] == INF:
            graph[i][j] = 0
for i in range(1, n+1):
    print(*graph[i][1:])

5
5
1 2 3
1 4 7
2 3 1
2 5 3
4 2 1
0 3 4 7 6
0 0 1 0 3
0 0 0 0 0
0 1 2 0 4
0 0 0 0 0


In [9]:
# 백준 1504번 : 특정한 최단 경로
n, m = map(int, input().split())
graph = [[] for _ in range(n+1)]
for i in range(m):
    a, b, c = map(int, input().rstrip().split())
    graph[a].append((b,c))
    graph[b].append((a,c))
v1, v2 = map(int, input().split())
INF = int(1e9)
import heapq
def shortest_path(start):
    distance = [INF]*(n+1)
    distance[start] = 0
    q = []
    heapq.heappush(q, (0, start))
    while q:
        dist, node = heapq.heappop(q)
        if dist > distance[node]:
            continue
        for x in graph[node]:
            if dist+x[1] < distance[x[0]]:
                distance[x[0]] = dist + x[1]
                heapq.heappush(q, (distance[x[0]], x[0]))
    return distance
start_path = shortest_path(1)
v1_path = shortest_path(v1)
v2_path = shortest_path(v2)
dist1 = start_path[v1]+v1_path[v2]+v2_path[n]
dist2 = start_path[v2]+v2_path[v1]+v1_path[n]
result = min(dist1, dist2)
if result >= INF:
    print(-1)
else:
    print(result)

4 1
1 2 2
2 3
-1


In [47]:
# 백준 9370번 : 미확인 도착지
import heapq
def shortest_path(start, graph, distance, visited):
    distance[start] = 0
    q = []
    heapq.heappush(q, (0, start))
    while q:
        dist, node = heapq.heappop(q)
        if dist > distance[node]:
            continue
        for x in graph[node]:
            # start부터 node까지의 거리 + node부터 x[0]까지의 거리
            temp = dist + x[1]
            if temp < distance[x[0]]:
                # start부터 node까지 갈 때 (g-h)를 통과하거나 node부터 x[0]까지가 (g-h)인 경우 >> 방문 처리
                if visited[node] == True or (node == g and x[0] == h) or (node == h and x[0] == g):
                    visited[x[0]] = True
                # 방문하지 않은 경우 >> 방문하지 않은 경우로 처리
                elif visited[node] == False:
                    visited[x[0]] = False
                distance[x[0]] = temp
                heapq.heappush(q, (temp, x[0]))
            # 최단거리가 여러개가 있는 경우 >> 방문 처리
            elif temp == distance[x[0]]:
                if visited[node] == True or (node == g and x[0] == h) or (node == h and x[0] == g):
                    visited[x[0]] = True
    return distance, visited    

num = int(input())
INF = int(1e9)
show = [[] for _ in range(num)]
for i in range(num):
    n, m, t = map(int, input().rstrip().split())
    start, g, h = map(int, input().rstrip().split())
    distance = [INF]*(n+1)
    graph = [[] for _ in range(n+1)]
    target = []
    visited = [False]*(n+1)
    for _ in range(m):
        a, b, c = map(int, input().rstrip().split())
        graph[a].append((b,c))
        graph[b].append((a,c))
    for _ in range(t):
        target.append(int(input()))
    result1, result2 = shortest_path(start, graph, distance, visited)
    for j in range(t):
        if result1[target[j]] >= INF or result2[target[j]] == False:
            pass
        else:
            show[i].append(target[j])
for i in range(num):
    print(*sorted(show[i]))

1
4 4 1
1 3 4
1 2 3
2 4 4
1 3 4
3 4 3
4
4
